# Build Simple CNN + MLP -> MLP Model

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential

2024-04-10 12:23:34.067568: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-10 12:23:34.089515: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-10 12:23:34.089534: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-10 12:23:34.090022: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-10 12:23:34.093907: I tensorflow/core/platform/cpu_feature_guar

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
train_table_df = pd.read_pickle("processed_data/GLC24_PA_metadata_train_elevation_onehot_grouped.pkl")
train_image_df_npz = np.load("processed_images/train_rgbnir.npz")
train_image_df = train_image_df_npz['arr_0']

In [4]:
train_table_df.head()

lon        lat  year    areaInM2  speciesId  elevation  ALPINE  \
surveyId                                                                        
212       3.099038  43.134956  2021  100.000000      11193       26.0       0   
222       9.884560  56.912140  2017   79.000000       9816       54.0       0   
243       8.256020  55.637050  2019   79.000000      10223       -2.0       0   
324      -0.402590  43.505630  2018  122.641821       9388      194.0       0   
333      -0.517360  45.806430  2017  122.641821      10247       54.0       0   

          ATLANTIC  BLACK SEA  BOREAL  CONTINENTAL  MEDITERRANEAN  PANNONIAN  \
surveyId                                                                       
212              0          0       0            0              1          0   
222              0          0       0            1              0          0   
243              1          0       0            0              0          0   
324              1          0       0            0              0          0   
333              1          0       0            0              0          0   

          STEPPIC  Andorra  Austria  Belgium  Bosnia and Herzegovina  \
surveyId                                                               
212             0        0        0        0                       0   
222             0        0        0        0                       0   
243             0        0        0        0                       0   
324             0        0        0        0                       0   
333             0        0        0        0                       0   

          Bulgaria  Croatia  Czech Republic  Denmark  France  Germany  Greece  \
surveyId                                                                        
212              0        0               0        0       1        0       0   
222              0        0               0        1       0        0       0   
243              0        0               0        1       0        0       0   
324              0        0               0        0       1        0       0   
333              0        0               0        0       1        0       0   

          Hungary  Ireland  Italy  Latvia  Luxembourg  Monaco  Montenegro  \
surveyId                                                                    
212             0        0      0       0           0       0           0   
222             0        0      0       0           0       0           0   
243             0        0      0       0           0       0           0   
324             0        0      0       0           0       0           0   
333             0        0      0       0           0       0           0   

          Netherlands  Norway  Poland  Portugal  Romania  Serbia  Slovakia  \
surveyId                                                                     
212                 0       0       0         0        0       0         0   
222                 0       0       0         0        0       0         0   
243                 0       0       0         0        0       0         0   
324                 0       0       0         0        0       0         0   
333                 0       0       0         0        0       0         0   

          Slovenia  Spain  Switzerland  \
surveyId                                 
212              0      0            0   
222              0      0            0   
243              0      0            0   
324              0      0            0   
333              0      0            0   

          The former Yugoslav Republic of Macedonia  2  3  4  7  9  10  12  \
surveyId                                                                     
212                                               0  0  0  0  0  0   0   0   
222                                               0  0  0  0  0  0   0   0   
243                                               0  0  0  0  0  0   0   0   
324                                      

In [5]:
# lon	lat	year	areaInM2	speciesId	elevation	ALPINE	ATLANTIC	BLACK SEA	BOREAL	CONTINENTAL	MEDITERRANEAN	PANNONIAN	STEPPIC	Andorra	Austria	Belgium	Bosnia and Herzegovina	Bulgaria	Croatia	Czech Republic	Denmark	France	Germany	Greece	Hungary	Ireland	Italy	Latvia	Luxembourg	Monaco	Montenegro	Netherlands	Norway	Poland	Portugal	Romania	Serbia	Slovakia	Slovenia	Spain	Switzerland	The former Yugoslav Republic of Macedonia
x_fields = ['lon', 'lat', 'year', 'areaInM2', 'elevation', 'ALPINE', 'ATLANTIC', 'BLACK SEA', 'BOREAL', 'CONTINENTAL', 'MEDITERRANEAN', 'PANNONIAN', 'STEPPIC', 'Andorra', 'Austria', 'Belgium', 'Bosnia and Herzegovina', 'Bulgaria', 'Croatia', 'Czech Republic', 'Denmark', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia', 'Luxembourg', 'Monaco', 'Montenegro', 'Netherlands', 'Norway', 'Poland', 'Portugal', 'Romania', 'Serbia', 'Slovakia', 'Slovenia', 'Spain', 'Switzerland', 'The former Yugoslav Republic of Macedonia']
len(x_fields)

X_table = train_table_df[x_fields]
Y = train_table_df.iloc[:,43:]

In [6]:
X_train_table, X_val_table, X_train_image, X_val_image, Y_train, Y_val = train_test_split(X_table, train_image_df, Y, test_size=0.2, shuffle=True, random_state=113)

In [7]:
# print shapes of X_train_table, X_val_table, X_train_image, X_image_image, Y_train, Y_val
print(X_train_table.shape)
print(X_val_table.shape)
print(X_train_image.shape)
print(X_val_image.shape)
print(Y_train.shape)
print(Y_val.shape)


(71189, 42)
(17798, 42)
(71189, 128, 128, 6)
(17798, 128, 128, 6)
(71189, 5016)
(17798, 5016)


In [8]:
Y_train.head()

2  3  4  7  9  10  12  14  15  16  18  24  28  29  30  32  35  37  \
surveyId                                                                      
2507016   0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0   
1084003   0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0   
204504    0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0   
1130935   0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0   
2774220   0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0   

          38  39  43  44  48  51  53  56  59  67  68  69  71  73  74  75  76  \
surveyId                                                                       
2507016    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
1084003    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
204504     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
1130935    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
2774220    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   

          77  78  80  81  82  87  88  91  92  96  100  101  102  106  107  \
surveyId                                                                    
2507016    0   0   0   0   0   0   0   0   0   0    0    0    0    0    0   
1084003    0   0   0   0   0   0   0   0   0   0    0    0    0    0    0   
204504     0   0   0   0   0   0   0   0   0   0    0    0    0    0    0   
1130935    0   0   0   0   0   0   0   0   0   0    0    0    0    0    0   
2774220    0   0   0   0   0   0   0   0   0   0    0    0    0    0    0   

          114  115  120  121  122  129  130  137  139  140  141  144  146  \
surveyId                                                                    
2507016     0    0    0    0    0    0    0    0    0    0    0    0    0   
1084003     0    0    0    0    0    0    0    0    0    0    0    0    0   
204504      0    0    0    0    0    0    0    0    0    0    0    0    0   
1130935     0    0    0    0    0    0    0    0    0    0    0    0    0   
2774220     0    0    0    0    0    1    0    0    0    0    0    0    0   

          147  148  149  151  154  156  159  164  167  169  171  174  178  \
surveyId                                                                    
2507016     0    0    0    0    0    0    0    0    0    0    0    0    0   
1084003     0    0    0    0    0    0    0    0    0    0    0    0    0   
204504      0    0    0    0    0    0    0    0    0    0    0    0    0   
1130935     0    0    0    0    0    0    0    0    0    0    0    0    0   
2774220     0    0    0    0    0    0    0    0    0    0    0    0    0   

          180  181  182  183  185  186  187  190  191  192  194  203  204  \
surveyId                                                                    
2507016     0    0    0    0    0    0    0    0    0    0    0    0    0   
1084003     0    0    0    0    0    0    0    0    0    0    0    0    0   
204504      0    0    0    0    0    0    0    0    0    0    0    0    1   
1130935     0    0    0    0    0    0    0    0    0    0    0    0    0   
2774220     0    0    0    0    0    0    0    0    0    0    0    0    0   

          205  207  209  215  216  218  220  222  224  225  226  232  236  \
surveyId                                                                    
2507016     0    0    0    0    0    0    0    0    0    0    0    0    0   
1084003     0    0    0    0    0    0    0    0    0    0    0    0    0   
204504      0    0    0    0    0    0    0    0    0    0    0    0    0   
1130935     0    0    0    0    0    0    0    0    0    0    0    0    0   
2774220     0    0    0    0    0    0    0    0    0    0    0    0    0   

          238  240  241  242  244  245  248  249  251  252  254  257  259  \
surveyId                                                                    
2507016     0    0    0    0    0    0   

## Image Only Model

## Tabular Data Model

### Combined Model

In [9]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model

# Define the Picture (CNN) Stream
input_pic = layers.Input(shape=(128, 128, 6))
x         = layers.Conv2D(128, (6, 6), activation='relu', input_shape=(128, 128, 6))(input_pic)
x         = layers.MaxPooling2D((2, 2))(x)
x         = layers.Conv2D(64, (3, 3), activation='relu')(x)
x         = layers.MaxPooling2D((2, 2))(x)
x         = layers.Conv2D(64, (3, 3), activation='relu')(x)
x         = layers.MaxPooling2D((5, 5))(x)
x         = layers.Flatten()(x)
x         = layers.Dense(200, activation='relu')(x)
x         = Model(inputs=input_pic, outputs=x)

# Define the Stats (Feed-Forward) Stream
input_stats = layers.Input(shape=(42,))
y = layers.Dense(128, activation="relu")(input_stats)
y = layers.Dense(200, activation="relu")(y)
y = Model(inputs=input_stats, outputs=y)

# Concatenate the two streams together
combined = layers.concatenate([x.output, y.output])

# Define joined Feed-Forward Layer
z = layers.Dense(400, activation="relu")(combined)
z = layers.Dense(400, activation="relu")(z)


# Define output node of 5016 neurons (classification task)
z = layers.Dense(5016, activation="softmax")(z)


# Define the final model
model = Model(inputs=[x.input, y.input], outputs=z)

2024-04-08 17:57:24.488483: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-08 17:57:24.568787: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-08 17:57:24.568813: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-08 17:57:24.586776: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-08 17:57:24.614316: I tensorflow/core/platform/cpu_feature_guar

In [10]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 6)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 123, 123, 128)        27776     ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 61, 61, 128)          0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv2d_1 (Conv2D)           (None, 59, 59, 64)           73792     ['max_pooling2d[0][0]'] 

In [11]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=0.001)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [12]:
# Create a model saving callback and train for 10 epochs (connect to GPU runtime!!)

from tensorflow.keras.callbacks import ModelCheckpoint

cp = ModelCheckpoint('model/', save_best_only=True)

model.fit(x=[X_train_image, X_train_table], y=Y_train, validation_data=([X_val_image, X_val_table], Y_val), epochs=10, callbacks=[cp])

Epoch 1/10


2024-04-08 17:57:32.459338: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-04-08 17:57:32.832414: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f43713fd820 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-08 17:57:32.832432: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-04-08 17:57:32.848365: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1712591852.972999   36405 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2225/2225 [==============================] - 36s 15ms/step - loss: nan - accuracy: 0.0039 - val_loss: nan - val_accuracy: 5.0567e-04
Epoch 2/10
2225/2225 [==============================] - 34s 15ms/step - loss: nan - accuracy: 3.0904e-04 - val_loss: nan - val_accuracy: 5.0567e-04
Epoch 3/10
2225/2225 [==============================] - 34s 15ms/step - loss: nan - accuracy: 3.0904e-04 - val_loss: nan - val_accuracy: 5.0567e-04
Epoch 4/10
2225/2225 [==============================] - 33s 15ms/step - loss: nan - accuracy: 3.0904e-04 - val_loss: nan - val_accuracy: 5.0567e-04
Epoch 5/10
2225/2225 [==============================] - 31s 14ms/step - loss: nan - accuracy: 3.0904e-04 - val_loss: nan - val_accuracy: 5.0567e-04
Epoch 6/10
2225/2225 [==============================] - 33s 15ms/step - loss: nan - accuracy: 3.0904e-04 - val_loss: nan - val_accuracy: 5.0567e-04
Epoch 7/10
2225/2225 [==============================] - 32s 15ms/step - loss: nan - accuracy: 3.0904e-04 - val_loss: nan - val_